- 下载数据

In [4]:
from numpy import*
def loadSimpData():
    datMat = matrix([[1.,  2.1],
                     [2.,  1.1],
                     [1.3,  1.],
                     [1.,  1.],
                     [2.,  1.]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat, classLabels

<font color='red'>
 
   - Numpy matrices必须是2维的,但是 numpy arrays (ndarrays) 可以是多维的（1D，2D，3D····ND）. Matrix是Array的一个小的分支，包含于Array。所以matrix 拥有array的所有特性。
    
   - 在numpy中matrix的主要优势是：相对简单的乘法运算符号。例如，a和b是两个matrices，那么a*b，就是矩阵积。
   -  matrix 和 array 都可以通过objects后面加.T 得到其转置。但是 matrix objects 还可以在后面加 .H f得到共轭矩阵, 加 .I 得到逆矩阵。

   

- 建立一个单层的决策树函数

In [5]:
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):  # just classify the data
    retArray = ones((shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
    return retArray

In [6]:
def buildStump(dataArr,classLabels,D):
    dataMatrix = mat(dataArr); labelMat = mat(classLabels).T
    m,n = shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = mat(zeros((m,1)))
    minError = inf   # init error sum, to +infinity
    for i in range(n):  # loop over all dimensions
        rangeMin = dataMatrix[:,i].min();  rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):  # over all range in current dimension
            for inequal in ['lt', 'gt']:   # go over less than and greater than
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)  # call stump classify with i, j, lessThan
                errArr = mat(ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr  #calc total error multiplied by D
                #print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClasEst


In [7]:
dataMat, classLables = loadSimpData()
D = mat(ones((5, 1))/5)
buildStump(dataMat, classLables, D)

({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'}, matrix([[0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

- Adaboost实现

<table border="1" width="100%" align="left">
  <tr>
    <th>序号</th>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
    <th>6</th>
    <th>7</th>
    <th>8</th>
    <th>9</th>
    <th>10</th>
  </tr>
  <tr>
    <td>x</td>
    <td>0</td>
    <th>1</th>
    <th>2</th>
    <th>3</th>
    <th>4</th>
    <th>5</th>
    <th>6</th>
    <th>7</th>
    <th>8</th>
    <th>9</th>
  </tr>
  <tr>
    <td>y</td>
    <td>1</td>
     <td>1</td>
      <td>1</td>
      <td>-1</td>
      <td>-1</td>
      <td>-1</td>
      <td>1</td>
      <td>1</td>
      <td>1</td>
      <td>-1</td>
  </tr>
  </table>
  
  
  我们以上表所示数据为例，来演示Adaboost的执行过程：
  初始化权值分布
  $$D_1=\left \{ w_{11},w_{12},w_{13},w_{14},w_{15},w_{16},w_{17},w_{18},w_{19},w_{110} \right \}$$
  $$w_{1i}=0.1\hspace{1cm}i=1,2\cdots10$$
  (a) 在权值为D1的训练数据上，阈值取2.5时分类器效果最好，故基本分类器为：
  $$G_{1}=\begin{cases}
  1&\text{x<2.5}\\
  -1&\text{x>2.5}
  \end{cases}$$

(b)$G_{1}(x)$在训练集上的误差率为$e_1=P(G_{1}(x_i)\neq y_{i})=0.3$

(c)计算$G_1(x)$的系数$\alpha_1=\frac{1}{2}\log\frac{1-e_1}{e_1}=0.4236$

(d)更新训练数据的权值分布：
$$D_2=\left \{ w_{21},w_{22},w_{23},w_{24},w_{25},w_{26},w_{27},w_{28},w_{29},w_{210} \right \}$$
$$w_{2i}=\frac{w_{1i}}{Z_1}exp(-\alpha_{1}y_iG_{1}(x_i))\hspace{1cm}i=1,2\cdots10$$
$$D_2=\left \{ 0.0715,0,0715,0.0715,0.0715,0.0715,0.0715,0.1666,0.1666,0.0715 \right \}$$
$$f_1(x)=0.4236G_1(x)$$

分类器$sign[f_1{x}]$在训练数据集上有三个误分类点。

对m=2

(a) 在权值为D2的训练数据上，阈值取8.5时分类器效果最好，故基本分类器为：
  $$G_{2}=\begin{cases}
  1&\text{x<8.5}\\
  -1&\text{x>8.5}
  \end{cases}$$
 (b)$G_{2}(x)$在训练集上的误差率为$e_2=P(G_{2}(x_i)\neq y_{i})=0.21413$
 
 (c)计算$G_2(x)$的系数$\alpha_2=\frac{1}{2}\log\frac{1-e_2}{e_2}=0.6496$
 
 (d)更新训练数据的权值分布：
 $$D_2=\left \{ 0.0455,0,0455,0.0455,0.1667,0.1667,0.1667,0.1060,0.1060,0.1060,0.0455 \right \}$$
 $$f_2(x)=0.4236G_1{x}+0.6496G_2(x)$$
 
 分类器$sign[f_2{x}]$在训练数据集上有三个误分类点。
 
 对m=3,
 
 (a) 在权值为D3的训练数据上，阈值取5.5时分类器效果最好，故基本分类器为：
  $$G_{3}=\begin{cases}
  1&\text{x<5.5}\\
  -1&\text{x>5.5}
  \end{cases}$$
  
  (b)$G_{3}(x)$在训练集上的误差率为$e_3=P(G_{3}(x_i)\neq y_{i})=0.1820$
  
  (c)计算$G_3(x)$的系数$\alpha_3=\frac{1}{2}\log\frac{1-e_3}{e_3}=0.7514$
  
  d)更新训练数据的权值分布：
 $$D_3=\left \{ 0.125,0.125,0.125,0.102,0.102,0.102,0.065,0.065,0.065,0.125 \right \}$$
 $$f_3(x)=0.4236G_1{x}+0.6496G_2(x)+0.7514G_3(x)$$
 
 分类器分类器$sign[f_3{x}]$在训练数据集上有零个误分类点。
 
 于是最终分类器为：
 $$G(x)=sign[f_3(x)=sign[0.4236G_1(x)+0.6496G_2(x)+0.7514G_3(x)]$$

In [8]:
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    weakClassArr = []
    m = shape(dataArr)[0]
    D = mat(ones((m,1))/m)   #init D to all equal
    aggClassEst = mat(zeros((m,1)))
    for i in range(numIt):
        bestStump,error,classEst = buildStump(dataArr,classLabels,D)#build Stump
        #print "D:",D.T
        alpha = float(0.5*log((1.0-error)/max(error,1e-16)))#calc alpha, throw in max(error,eps) to account for error=0
        bestStump['alpha'] = alpha  
        weakClassArr.append(bestStump)                  #store Stump Params in Array
        #print "classEst: ",classEst.T
        expon = multiply(-1*alpha*mat(classLabels).T,classEst) #exponent for D calc, getting messy
        D = multiply(D,exp(expon))                              #Calc New D for next iteration
        D = D/D.sum()
        #calc training error of all classifiers, if this is 0 quit for loop early (use break)
        aggClassEst += alpha*classEst
        #print "aggClassEst: ",aggClassEst.T
        aggErrors = multiply(sign(aggClassEst) != mat(classLabels).T,ones((m,1)))
        errorRate = aggErrors.sum()/m
        print("total error: ",errorRate)
        if errorRate == 0.0: break
    return weakClassArr


In [9]:
def adaClassify(datToClass,classifierArr):
    dataMatrix = mat(datToClass)#do stuff similar to last aggClassEst in adaBoostTrainDS
    m = shape(dataMatrix)[0]
    aggClassEst = mat(zeros((m,1)))
    for i in range(len(classifierArr)):
        classEst = stumpClassify(dataMatrix, classifierArr[i]['dim'],\
                                 classifierArr[i]['thresh'],\
                                 classifierArr[i]['ineq'])#call stump classify
        aggClassEst += classifierArr[i]['alpha']*classEst
        print(aggClassEst)
    return sign(aggClassEst)

In [10]:
datArr, labelArr = loadSimpData()
classifierArr = adaBoostTrainDS(datArr, labelArr, 30)
adaClassify([0, 0], classifierArr)

total error:  0.2
total error:  0.2
total error:  0.0
[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])